# Dallas Cowboys Season 2021 stats

In [48]:
import pandas as pd

## Data Extraction

In [49]:
dallas_url = "https://www.pro-football-reference.com/teams/dal/2021.htm"

In [50]:
dfs = pd.read_html(dallas_url)
season_stats_raw = dfs[0]
weekly_stats_raw = dfs[1]
redzone_downs_raw = dfs[0]

# Data Cleaning

In [51]:
print(f"Shape: {weekly_stats_raw.shape}")
weekly_stats_raw.info()

Shape: (20, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 25 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   (Unnamed: 0_level_0, Week)                19 non-null     object 
 1   (Unnamed: 1_level_0, Day)                 18 non-null     object 
 2   (Unnamed: 2_level_0, Date)                19 non-null     object 
 3   (Unnamed: 3_level_0, Unnamed: 3_level_1)  18 non-null     object 
 4   (Unnamed: 4_level_0, Unnamed: 4_level_1)  18 non-null     object 
 5   (Unnamed: 5_level_0, Unnamed: 5_level_1)  18 non-null     object 
 6   (Unnamed: 6_level_0, OT)                  2 non-null      object 
 7   (Unnamed: 7_level_0, Rec)                 18 non-null     object 
 8   (Unnamed: 8_level_0, Unnamed: 8_level_1)  9 non-null      object 
 9   (Unnamed: 9_level_0, Opp)                 19 non-null     object 
 10  (Score, Tm)             

In [52]:
weekly_stats_raw.dtypes

Unnamed: 0_level_0  Week                   object
Unnamed: 1_level_0  Day                    object
Unnamed: 2_level_0  Date                   object
Unnamed: 3_level_0  Unnamed: 3_level_1     object
Unnamed: 4_level_0  Unnamed: 4_level_1     object
Unnamed: 5_level_0  Unnamed: 5_level_1     object
Unnamed: 6_level_0  OT                     object
Unnamed: 7_level_0  Rec                    object
Unnamed: 8_level_0  Unnamed: 8_level_1     object
Unnamed: 9_level_0  Opp                    object
Score               Tm                    float64
                    Opp                   float64
Offense             1stD                  float64
                    TotYd                 float64
                    PassY                 float64
                    RushY                 float64
                    TO                    float64
Defense             1stD                  float64
                    TotYd                 float64
                    PassY                 float64


In [53]:
# Create column new names dict
column_dict = {
    "TotYd": "Total_Yards",
    "1stD":  "First_Downs",
    "TO": "Turnovers",
    "RushY": "Rush_Yards",
    "PassY": "Pass_Yards",
    "Rec": "Record",
    "Score": "Points",
    "Tm": "Scored",
    "Opp": "Allowed",
    "Unnamed: 0_level_0": "",
    "Unnamed: 1_level_0": "",
    "Unnamed: 2_level_0": "",
    "Unnamed: 3_level_0": "",
    "Unnamed: 4_level_0": "",
    "Unnamed: 5_level_0": "",
    "Unnamed: 6_level_0": "",
    "Unnamed: 7_level_0": "",
    "Unnamed: 8_level_0": "",
    "Unnamed: 9_level_0": "",
    "Unnamed: 3_level_1": "Kickoff_Time",
    "Unnamed: 4_level_1": "Boxscore",
    "Unnamed: 5_level_1": "Result",
    "Unnamed: 8_level_1": "Local",
    "Sp. Tms": "Special_Teams"    
}
# Apply names to columns
weekly_stats_raw = weekly_stats_raw.rename(columns=column_dict)
# Create list of columns
week_new_columns = [' '.join(col).strip() for col in weekly_stats_raw.columns.values]
week_new_columns = [col_name.replace(" ", "_") for col_name in week_new_columns]
week_new_columns = ["Opponent" if col_name == "Allowed" else col_name for col_name in week_new_columns]
# Apply new column names (flatten)
weekly_stats_raw.columns = week_new_columns

In [54]:
weekly_stats_clean = weekly_stats_raw.copy(deep=True)

In [55]:
# Remove empty rows
weekly_stats_clean = weekly_stats_clean[weekly_stats_clean['Day'].notna()]

In [56]:
# Add years and hour to date
weekly_stats_clean.loc[weekly_stats_clean["Week"].str.isnumeric(), "Date"] += " 2021 " + weekly_stats_clean.loc[weekly_stats_clean["Week"].str.isnumeric(), "Kickoff_Time"]
weekly_stats_clean.loc[weekly_stats_clean["Week"] == "Wild Card", "Date"] += " 2022 " + weekly_stats_clean.loc[weekly_stats_clean["Week"] == "Wild Card", "Kickoff_Time"] 
# Transform date str to datetime
weekly_stats_clean["Date"] = pd.to_datetime(weekly_stats_clean["Date"], format='%B %d %Y %I:%M%p ET')
weekly_stats_clean["Date"][0]

Timestamp('2021-09-09 20:20:00')

In [57]:
weekly_stats_clean.drop(columns=["Boxscore","Kickoff_Time","Day","Expected_Points_Offense", "Expected_Points_Defense",
    "Expected_Points_Special_Teams"], inplace=True)
weekly_stats_clean.columns

Index(['Week', 'Date', 'Result', 'OT', 'Record', 'Local', 'Opponent',
       'Points_Scored', 'Points_Allowed', 'Offense_First_Downs',
       'Offense_Total_Yards', 'Offense_Pass_Yards', 'Offense_Rush_Yards',
       'Offense_Turnovers', 'Defense_First_Downs', 'Defense_Total_Yards',
       'Defense_Pass_Yards', 'Defense_Rush_Yards', 'Defense_Turnovers'],
      dtype='object')

In [58]:
# Replace missing values with 0's
weekly_stats_clean.fillna(0, inplace=True)

In [65]:
# Modify columns data
weekly_stats_clean["OT"] = weekly_stats_clean["OT"].map(lambda x: True if x=="OT" else False)
weekly_stats_clean["Result"] = weekly_stats_clean["Result"].map(lambda x: True if x=="W" else False)
weekly_stats_clean["Local"] = weekly_stats_clean["Local"].map(lambda x: True if x!="@" else False)
weekly_stats_clean["Local"]

0     0.87
1     0.53
2     0.58
3     0.43
4     0.61
5     0.78
7     0.81
8     0.73
9     0.74
10    0.70
11    0.85
12    0.61
13    0.62
14    0.62
15    0.78
16    0.85
17    0.64
19    0.75
Name: Offense_Pct_Pass, dtype: float64

In [84]:
def get_team_type(total_yards: int, pass_yards: int) -> str:
    pass_pct = pass_yards/total_yards
    if pass_pct > (2/3):
        team_type = "Passing"
    elif pass_pct > (1/3):
        team_type = "Balanced"
    else:
        team_type = "Rushing"
    return team_type

weekly_stats_clean['Offense_Type'] = weekly_stats_clean.apply(lambda x: get_team_type(x['Offense_Total_Yards'], x['Offense_Pass_Yards']), axis=1)
weekly_stats_clean['Opponent_Type'] = weekly_stats_clean.apply(lambda x: get_team_type(x['Defense_Total_Yards'], x['Defense_Pass_Yards']), axis=1)

In [60]:
reg_season_stats = weekly_stats_clean.iloc[:-1,:].convert_dtypes()
postseason_stats = weekly_stats_clean.iloc[-1,:].to_frame().convert_dtypes()

### Data Exploration

#### Regular Season Stats

In [61]:
reg_season_stats.describe()

,Result,OT,Local,Points_Scored,Points_Allowed,Offense_First_Downs,Offense_Total_Yards,Offense_Pass_Yards,Offense_Rush_Yards,Offense_Turnovers,Defense_First_Downs,Defense_Total_Yards,Defense_Pass_Yards,Defense_Rush_Yards,Defense_Turnovers,Offense_Pct_Rush,Offense_Pct_Pass,Defense_Pct_Rush,Defense_Pct_Pass
count,17,17,17,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.0000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
unique,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,12,15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,31.176471,21.058824,22.941176,407.000000,282.352941,124.647059,1.176471,19.235294,351.0000,238.176471,112.823529,2.000000,0.308824,0.691176,0.333529,0.666471
std,NaN,NaN,NaN,12.870680,8.735492,4.904979,83.432608,82.732960,56.083577,0.882843,4.981584,78.1193,78.413356,34.912812,1.457738,0.124694,0.124694,0.108048,0.108048
min,NaN,NaN,NaN,9.000000,3.000000,14.000000,276.000000,188.000000,45.000000,0.000000,11.000000,214.0000,111.000000,52.000000,0.000000,0.130000,0.430000,0.120000,0.520000
25%,NaN,NaN,NaN,21.000000,17.000000,19.000000,328.000000,212.000000,78.000000,1.000000,15.000000,302.0000,177.000000,95.000000,1.000000,0.220000,0.610000,0.280000,0.590000
50%,NaN,NaN,NaN,29.000000,20.000000,23.000000,419.000000,256.000000,122.000000,1.000000,20.000000,367.0000,244.000000,113.000000,2.000000,0.300000,0.700000,0.340000,0.660000
75%,NaN,NaN,NaN,41.000000,28.000000,26.000000,451.000000,341.000000,160.000000,2.000000,23.000000,405.0000,294.000000,127.000000,3.000000,0.390000,0.780000,0.410000,0.720000


##### Comparación victorias y derrotas

In [62]:
reg_season_stats.columns

Index(['Week', 'Date', 'Result', 'OT', 'Record', 'Local', 'Opponent',
       'Points_Scored', 'Points_Allowed', 'Offense_First_Downs',
       'Offense_Total_Yards', 'Offense_Pass_Yards', 'Offense_Rush_Yards',
       'Offense_Turnovers', 'Defense_First_Downs', 'Defense_Total_Yards',
       'Defense_Pass_Yards', 'Defense_Rush_Yards', 'Defense_Turnovers',
       'Offense_Pct_Rush', 'Offense_Pct_Pass', 'Defense_Pct_Rush',
       'Defense_Pct_Pass'],
      dtype='object')

In [83]:
reg_season_stats.loc[reg_season_stats["Offense_Turnovers"] > reg_season_stats["Defense_Turnovers"]]

,Week,Date,Result,OT,Record,Local,Opponent,Points_Scored,Points_Allowed,Offense_First_Downs,...,Offense_Turnovers,Defense_First_Downs,Defense_Total_Yards,Defense_Pass_Yards,Defense_Rush_Yards,Defense_Turnovers,Offense_Pct_Rush,Offense_Pct_Pass,Defense_Pct_Rush,Defense_Pct_Pass
7,8,2021-10-31 20:20:00,True,False,6-1,False,Minnesota Vikings,20,16,23,...,2,17,278,177,101,0,0.19,0.81,0.36,0.64
8,9,2021-11-07 13:00:00,False,False,6-2,True,Denver Broncos,16,30,14,...,2,23,407,217,190,0,0.27,0.73,0.47,0.53
10,11,2021-11-21 16:25:00,False,False,7-3,False,Kansas City Chiefs,9,19,16,...,3,22,370,244,126,2,0.3,0.7,0.34,0.66
16,17,2021-01-02 13:00:00,False,False,11-5,True,Arizona Cardinals,22,25,19,...,1,23,399,272,127,0,0.15,0.85,0.32,0.68


In [76]:
results_df = reg_season_stats.groupby("Result").agg({
    "Points_Scored": "mean",
    "Points_Allowed": "mean",
    "Offense_Total_Yards": ["mean", "median"], # skewed
    "Offense_Pass_Yards": ["mean", "median"], # skewed
    "Offense_Rush_Yards": ["mean", "median"], # skewed
    "Defense_Total_Yards": ["mean", "median"], # skewed
    "Defense_Pass_Yards": ["mean", "median"], # skewed
    "Defense_Rush_Yards": ["mean", "median"], # skewed
    "Offense_Turnovers": ["mean", "median", "min", "max", "sum"],
    "Defense_Turnovers": ["mean", "median", "min", "max", "sum"]
}).round(2)
#["mean", "median"]
results_df

Points_Scored Points_Allowed Offense_Total_Yards         \
                mean           mean                mean median   
Result                                                           
False           21.8           28.2               351.0  301.0   
True           35.08          18.08              430.33  425.0   

       Offense_Pass_Yards        Offense_Rush_Yards         \
                     mean median               mean median   
Result                                                       
False               285.2  256.0               65.8   64.0   
True               281.17  267.5             149.17  135.5   

       Defense_Total_Yards         ... Offense_Turnovers                     \
                      mean median  ...              mean median min max sum   
Result                             ...                                        
False                423.2  407.0  ...               1.4    1.0   0   3   7   
True                320.92  325.0  ...              1.08    1.0   0   2  13   

       Defense_Turnovers                     
                    mean median min max sum  
Result                                       
False                1.2    0.0   0   4   6  
True                2.33    2.0   0   4  28  

[2 rows x 24 columns]

In [64]:
stadium_df = reg_season_stats.groupby("Local").agg({
    "Points_Scored": "mean",
    "Points_Allowed": "mean"
}).round(2)
stadium_df

,Points_Scored,Points_Allowed
Local,,
False,26.56,20.11
True,36.38,22.12
